In [1]:
from pathlib import Path

import pandas as pd

In [2]:
# read master sequestration file from ~/wd/master_sequestration_locations_72180_2023-04-25.tsv file
master_sequestration_file = Path.home() / "wd" / "master_sequestration_locations_72180_2023-04-25.tsv"
master_sequestration = pd.read_csv(master_sequestration_file, sep="\t")
open_ids = master_sequestration.loc[master_sequestration["dataset"] == "Open", "case_ids"]

In [3]:
# read all files from ~/dicom/files/ and create a dataframe for each file in a dictionary
manifests_list = list(Path("~/dicom/manifests/").expanduser().glob("**/*.tsv"))
# filter only ones that match this name "new_image_manifest_RSNA"
manifests_list = [x for x in manifests_list if x.name.startswith("image_manifest_")]
display(manifests_list)

[PosixPath('/home/ubuntu/dicom/manifests/image_manifest_ACR_20220606.tsv')]

In [4]:
dfs = {f.name: pd.read_csv(f, sep="\t") for f in manifests_list}

In [5]:
# filter dfs by case_ids from master_sequestration DataFrame
new_dfs = {k: v[v["case_ids"].isin(open_ids)] for k, v in dfs.items()}

In [6]:
for k in dfs:
    print(k, dfs[k].shape[0], new_dfs[k].shape[0])

image_manifest_ACR_20220606.tsv 5029 1884


In [ ]:
new_dfs

In [8]:
# new_dfs["new_image_manifest_RSNA_20230324.tsv"]
# new_dfs["image_manifest_TCIA-COVID-19-NY-SBU.tsv"]['storage_urls']
new_dfs["image_manifest_ACR_20220606.tsv"]['storage_urls']

1       /acrimage/2021/0827/2.16.840.1.114274.1818.536...
2       /acrimage/2021/0827/2.16.840.1.114274.1818.536...
3       /acrimage/2021/0827/2.16.840.1.114274.1818.536...
4       /acrimage/2021/0827/2.16.840.1.114274.1818.536...
5       /acrimage/2021/0827/2.16.840.1.114274.1818.536...
                              ...                        
5022    /ACR_20220314/10008204-a0vQKtFN0mm5Dp7SvAFw/2....
5023    /ACR_20220314/10008204-cWL2t6c0u0WemOPUuJGNg/2...
5025    /ACR_20220218/2.16.840.1.114274.1818.506608849...
5026    /ACR_20220218/2.16.840.1.114274.1818.499943339...
5027    /ACR_20220218/2.16.840.1.114274.1818.560063645...
Name: storage_urls, Length: 1884, dtype: object

In [9]:
for k, v in new_dfs.items():
    tmp = v

    if tmp["storage_urls"].starstswith("/ACR_20220314/"):
        tmp["storage_urls"].str.replace("/ACR_20220314/", "").to_csv(Path.home() / "dicom/manifests/processed" / "processed_{}".format(k), sep="\t", index=False, header=False)
    elif tmp["storage_urls"].starstswith("/ACR_20220218/"):
        tmp["storage_urls"].str.replace("/ACR_20220218/", "").to_csv(Path.home() / "dicom/manifests/processed" / "processed_{}".format(k), sep="\t", index=False, header=False)
    elif tmp["storage_urls"].starstswith("/acrimage/2021/0827/"):
        tmp["storage_urls"].str.replace("/acrimage/2021/0827/", "").to_csv(Path.home() / "dicom/manifests/processed" / "processed_{}".format(k), sep="\t", index=False, header=False)
    else:
        tmp["storage_urls"].str.replace("s3://storage.ir.rsna.ai/", "").to_csv(Path.home() / "dicom/manifests/processed" / "processed_{}".format(k), sep="\t", index=False, header=False)
    # tmp["storage_urls"].str.replace("storage.ir.rsna.ai", "external-data-midrc-replication")
    # For TCIA:
    # tmp["storage_urls"].str.replace("s3://midrcprod-default-813684607867-upload/", "").to_csv(Path.home() / "dicom/manifests/processed" / "processed_{}".format(k), sep="\t", index=False, header=False)


AttributeError: 'Series' object has no attribute 'starstswith'